In [1]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import datetime
import re
import requests
import pandas as pd

In [2]:
class Get_list_of_items:
    def __init__(self,types,cl,response):
        self.types,self.cl= types,cl
        self.html = BeautifulSoup(response.content, "html.parser")
    def lst(self, s = False):
        items = self.html.find_all(self.types, self.cl)
        lst = [i.text for i in items]
        return lst
        
class Get_list_of_reviews(Get_list_of_items):
    def lst_for_review_body(self,a):
        reviews = []
        for review in self.html.find_all('div', class_='review_body'):
            if review.find('span') is None:
                print(a)
            elif review.find('span', class_='inline_expand_collapse'):
                # Expanded review
                if review.find('span', class_='blurb_expanded') is None:
                    print(a)
                else:
                    expanded_review = review.find('span', class_='blurb_expanded').text.strip().replace('\r', '').replace('\n', '').replace('\\', '')
                    reviews.append(expanded_review)
            else:
                # Short review
                short_review = review.find('span').text.strip().replace('\r', '').replace('\n', '').replace('\\', '')
                reviews.append(short_review)
        return '. '.join(reviews)
        
    def lst(self,href = False, text = None):
        items = self.html.find_all(self.types, self.cl,href = href, text = text)
        lst = [i['href'] for i in items]
        return lst
    def get_users_reviews_url(self,url,page):
        response = requests.get(url,headers={'user-agent': 'My app'})
        if response.status_code != 200:
            raise TypeError('Response not 200')
        href_value = Get_list_of_reviews('a', None,response)
        href_link = href_value.lst(href = True,text='USER SCORE')
        return f"https://www.metacritic.com{href_link[0]}?page={page}"
    def page_doesnt_exist(self,url):
        response = requests.get(url,headers={'user-agent': 'My app'})
        if response.status_code != 200:
            raise TypeError('Response not 200')
        href_value = Get_list_of_items('div', "pad_top1 pad_btm2",response)
        href_link = href_value.lst(s = True)
        if not href_link:
            return False
        else:
            return True
        
    def lst_reviews(self):
        titles = self.lst(href = True)
        reviews_lst_by_title = []
        
        for a in tqdm(titles):
            reviews_lst_by_page = []
            page = 0
            while True:
                url = self.get_users_reviews_url(f"https://www.metacritic.com{a}",page)
                if self.page_doesnt_exist(url):
                    break
                response = requests.get(url,headers={'user-agent': 'My app'})
                if response.status_code != 200:
                    raise TypeError('Response not 200')
                reviews = Get_list_of_reviews('div', 'review_body',response)
                reviews_lst = reviews.lst_for_review_body(a)
                reviews_lst_by_page.append(reviews_lst)
                page += 1
            reviews_lst_by_title.append('. '.join(reviews_lst_by_page))
        return reviews_lst_by_title
    
        
        
        

def validate(date_text):
    try:
        datetime.datetime.strptime(date_text, '%B %d, %Y')
    except ValueError:
        raise TypeError('not a date')
def parse_games():
    df = pd.DataFrame(columns=['title','score','platform','date','summary','user_reviews'])
    for page in tqdm(range(200)):
        url = f'https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?page={page}'
        response = requests.get(url,headers={'user-agent': 'My app'})
        html = BeautifulSoup(response.content, "html.parser")
        if response.status_code != 200:
            raise TypeError('Response not 200')
        # Получаем список названий
        titles = Get_list_of_items('a', 'title',response)
        lst_titles = titles.lst()
        # Получаем список оценок
        score_wraper = html.find_all('div', class_ = 'clamp-metascore')
        lst_scores = [i.find('div', ['metascore_w large game positive','metascore_w large game mixed','metascore_w large game negative']).text for i in score_wraper]
        # Получаем список платформ
        platforms_row = html.find_all('div', class_ = 'platform')
        lst_platforms = [i.find('span', class_ = 'data').text.replace("\n", "").strip() for i in platforms_row if i.find('span', class_ = 'data') != None]
        # Получаем список дат
        dates = Get_list_of_items('div', 'clamp-details',response)
        lst_dates = [re.search('(?<=\\n\\n)([\w]+[\s]+[\d]+[\,]+[\s][\d]+)', i)[0] for i in dates.lst()]
        next(map(validate, lst_dates))
        # Получаем список описания
        summary = Get_list_of_items('div', 'summary',response)
        lst_summary = [i.replace("\n", "").strip() for i in summary.lst()]
        lst_summary = [re.sub('\[(.*?)\]','', i).strip() for i in lst_summary]
         # получаем список обзоров пользователей
        reviews = Get_list_of_reviews('a', 'title',response)
        reviews_by_title = reviews.lst_reviews()
        df1 = pd.DataFrame({'title': lst_titles, 
                            'score': lst_scores, 
                            'platform':lst_platforms,
                            'date':lst_dates,
                            'summary':lst_summary, 
                            'user_reviews': lst_reviews})
        df =pd.concat([df, df1])
    return df.reset_index(drop=True)
def parse_movies():
    df = pd.DataFrame(columns=['title','score','date','summary','user_reviews'])
    for page in tqdm(range(153)):
        url = f'https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page={page}'
        response = requests.get(url,headers={'user-agent': 'My app'})
        html = BeautifulSoup(response.content, "html.parser")
        if response.status_code != 200:
            raise TypeError('Response not 200')
        # Получаем список названий
        titles = Get_list_of_items('a', 'title',response)
        lst_titles = titles.lst()
        # Получаем список оценок
        score_wraper = html.find_all('div', class_ = 'clamp-metascore')
        lst_scores = [i.find('div', ['metascore_w large movie positive perfect','metascore_w large movie positive', 'metascore_w large movie negative','metascore_w large movie mixed']).text for i in score_wraper]
        # Получаем список дат
        dates = Get_list_of_items('div', 'clamp-details',response)
        lst_dates = [re.search('(?<=\\n)\S(.*?)(?=\\n)', i)[0] for i in dates.lst()]
        next(map(validate, lst_dates))
        # Получаем список описания
        summary = Get_list_of_items('div', 'summary',response)
        lst_summary = [i.replace("\n", "").strip() for i in summary.lst()]
        lst_summary = [re.sub('\[(.*?)\]','', i).strip() for i in lst_summary]
        # получаем список обзоров пользователей
        reviews = Get_list_of_reviews('a', 'title',response)
        reviews_by_title = reviews.lst_reviews()
#         print(reviews_by_title)
        df1 = pd.DataFrame({'title': lst_titles, 
                            'score': lst_scores,
                            'date':lst_dates,
                            'summary':lst_summary, 
                            'user_reviews': reviews_by_title })
        df =pd.concat([df, df1])
        df.to_csv('movies_dataset_with_reviews.csv',index=False)
    return df.reset_index(drop=True)
df = parse_movies()

  0%|                                                                                          | 0/153 [02:33<?, ?it/s]


KeyboardInterrupt: 

In [12]:
df = parse_games()

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [18:47<00:00,  5.64s/it]


In [13]:
df.to_csv('movies_dataset.csv',index=False)

,title,score,platform,date,summary
0,The Legend of Zelda: Ocarina of Time,99,Nintendo 64,"November 23, 1998","As a young boy, Link is tricked by Ganondorf, ..."
1,Tony Hawk's Pro Skater 2,98,PlayStation,"September 20, 2000",As most major publishers' development efforts ...
2,Grand Theft Auto IV,98,PlayStation 3,"April 29, 2008",What does the American Dream mean today? For N...
3,SoulCalibur,98,Dreamcast,"September 8, 1999","This is a tale of souls and swords, transcendi..."
4,Grand Theft Auto IV,98,Xbox 360,"April 29, 2008",What does the American Dream mean today? For N...
...,...,...,...,...,...
19992,Vroom in the Night Sky,17,Switch,"April 5, 2017",Vroom in the night sky is a magical bike actio...
19993,Leisure Suit Larry: Box Office Bust,17,PlayStation 3,"May 5, 2009",The Leisure Suit Larry: Box Office Bust video ...
19994,Yaris,17,Xbox 360,"October 10, 2007",Hop into a Toyota Yaris and drive it on crazy ...
19995,Ride to Hell: Retribution,16,PC,"June 24, 2013",The game is set in the last years of the roari...


In [3]:
df = pd.read_csv(r'C:\Users\misha\Pet-project\Parsing\movies_dataset (1).csv')
df.loc[df['title'] == 'Spider-man 2']

,title,score,date,summary,user_reviews
0,Tokyo Story,100,"March 13, 1972",Yasujiro Ozu’s Tokyo Story follows an aging co...,Perhaps the greatness of Tokyo Story lies in t...
1,The Godfather,100,"March 24, 1972",Francis Ford Coppola's epic features Marlon Br...,"Like Babe Ruth is to baseball, like John Stein..."
2,Citizen Kane,100,"September 4, 1941","Following the death of a publishing tycoon, ne...","Outstanding film, no question.'Citizen Kane', ..."
3,Rear Window,100,"September 1, 1954",A wheelchair-bound photographer spies on his n...,Jeff Jefferies is a photographer who likes to ...
4,Casablanca,100,"January 23, 1943","A Casablanca, Morocco casino owner in 1941 she...",“Casablanca” is a wonderful movie. For refere...
...,...,...,...,...,...
1295,Three Kings,82,"October 1, 1999",A small group of adventurous American soldiers...,I just watched this for the second time (2006)...
1296,The Northman,82,"April 22, 2022",From visionary director Robert Eggers comes Th...,Robert Eggers is 3 for 3. A masterpiece in eve...
1297,Neruda,82,"December 16, 2016",Beloved poet Pablo Neruda (Luis Gnecco) is als...,"An inventive, semi-surrealistic chronicle of t..."
1298,Blancanieves,82,"March 29, 2013",Once upon a time there was a little girl who h...,"The title means Snow White in Spanish, so ther..."


In [7]:
df.loc[df['title'] == 'Spider-Man 2']['user_reviews'][1022]

'Spider-Man 2 may be called a comic book movie, but is so much more than that. The film improves the action thanks to a larger budget, has awesome acting and characters, and an excellent script which has a heart and entertains. Spider-Man 2 is a movie you truly have to see to believe, it is the master of comic book movies along with The Dark Knight. Even if you are not a Spider-Man fan, SEE THIS MOVIE!. Spider-Man 2 is some of the most fun I have had during a movie. It is much more than a comic book movie and the character development in this movie is just great. I cared much more for the characters in this movie than Spider-Man (2002). The action sequences are brilliant in this movie, especially the train battle between our friendly-neighborhood Spider-Man and Doctor Octopus. Mary Jane is a forceful character in this movie though and by that, I mean, we are forced to like her as a character. She wasn\'t nearly as charming as Emma Stone was in The Amazing Spider-Man (2012) but this sti